In [16]:
%pip install scikit-learn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached scikit_learn-1.2.2-cp39-cp39-macosx_10_9_x86_64.whl (9.1 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring

In [80]:
import os
import re
import pandas as pd
import random

data = pd.DataFrame()
folder_path = "/Users/gauthamreddy/Desktop/Competition Project/data/TrainingData"
session_set = set()

for filename in os.listdir(folder_path):
    match = re.search(r'subject_(\d+)_(\d+)__(\w+)\.csv', filename)
    
    if match:
        session_set.add((match.group(1), match.group(2)))



for session in session_set:
    x_data = pd.read_csv(folder_path + '/subject_{}_{}__{}.csv'.format(session[0], session[1], 'x'))
    x_time = pd.read_csv(folder_path + '/subject_{}_{}__{}.csv'.format(session[0], session[1], 'x_time'))
    y_data = pd.read_csv(folder_path + '/subject_{}_{}__{}.csv'.format(session[0], session[1], 'y'))
    y_time = pd.read_csv(folder_path + '/subject_{}_{}__{}.csv'.format(session[0], session[1], 'y_time'))

    # set the time column as the index
    x_time.columns = ['time']
    y_time.columns = ['time']
    x_time['time'] = pd.to_datetime(x_time['time'], unit='s', origin='1970-01-01')
    y_time['time'] = pd.to_datetime(y_time['time'], unit='s', origin='1970-01-01')
    x_time.set_index('time', inplace=True)
    y_time.set_index('time', inplace=True)

    # resample x_data and y_data to the same rate
    resampled_x_data = x_data.set_index(x_time.index).resample('0.1S').pad()
    resampled_y_data = y_data.set_index(y_time.index).resample('0.1S').pad()

    # merge the resampled dataframes
    merged_df = pd.concat([resampled_x_data, resampled_y_data], axis=1)
    merged_df = merged_df.dropna().reset_index(drop=True)
    print(merged_df.to_numpy)



print(data.shape)

<bound method DataFrame.to_numpy of        1.524319e+00  9.704319e+00  1.856808e-01  -6.483540e-03  1.111111e-03  \
0          1.480000      9.700000      0.194575      -0.001284      0.001198   
1          1.543130      9.685623      0.178753       0.004168      0.004030   
2          1.480733      9.690733      0.173901       0.000000     -0.016836   
3          1.509605      9.673202      0.133202      -0.002499     -0.001388   
4          1.616597      9.733361      0.226723       0.004186      0.006744   
...             ...           ...           ...            ...           ...   
13732      2.638495      8.930000      2.779911       0.050353      0.033444   
13733      2.565741      9.201704      2.750000       0.016135     -0.005227   
13734      2.727663      9.064547      2.919221       0.011624      0.007371   
13735      2.646869      9.229478      2.782609      -0.016978     -0.015570   
13736      2.382506      9.074127      2.927494      -0.035880     -0.018293   

   

In [68]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

# Split data into inputs (X) and output (y)
X = data.iloc[:, 0:6].values
y = data.iloc[:, 6].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create random forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf.fit(X_train, y_train)

# Predict the output for the test data
y_pred = rf.predict(X_test)

# Evaluate the model using R-squared score
r2 = r2_score(y_test, y_pred)
print('R-squared score:', r2)


IndexError: single positional indexer is out-of-bounds